# Konvertering fra File Geodatabase (GDB) til FlatGeobuf (FGB)
Denne notebooken viser hvordan du kan konvertere data fra Kartverkets N50-kartdata (gdb-format) til FlatGeobuf (fgb), et åpent og effektivt dataformat egnet for moderne utviklings- og webmiljøer.

## Hvorfor FlatGeobuf?
FlatGeobuf (FGB) gir flere fordeler sammenlignet med det tradisjonelle File Geodatabase-formatet (GDB):

- Åpent, lisensfritt og godt støttet i både Python, JavaScript og moderne GIS-verktøy.
- Én fil per datasett gjør deling, versjonskontroll og distribusjon enkelt.
- Rask innlesing, effektiv filstruktur og støtte for spatial indexering.
- Utviklervennlig og lett å bruke – basert på åpne formater uten lukkede lisenser

*NB: Dette betyr ikke at man skal unngå å benytte GDB, som i stor grad legger til rette for mange andre verdifulle muligheter, men FGB kan anses som mer utviklervennlig.*

Denne notebooken gjør det enkelt å konvertere GDB-lag til FGB og kan utvides etter behov.

---

### Steg 1: Last ned kartdatene for N50
1. Naviger til Geonorge.no og finn [kartdataene for N50](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac), eller klikk på denne lenken - https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac

2. Her klikker du på valgmuligheten; "Last ned". N50 datasettet har nå blitt lagt til på Geonorges nedlastningside.
    # <img src="../../public/img/geonorge-n50.png" width="1200" alt="Geonorge N50 kartdata">

3.  Til denne demoene brukes:
    - `Hele landet`
    - `UTM 33`
    - `FGBD`
    
    Fyll inn informasjons i skjemaet og trykk `Last ned`. Du skal nå motta en zip-fil
    # <img src="../../public/img/geonorge-lastned.png" width="1200" alt="Geonorge nedlastingsside">

4. Legg til en ny `data` mappe i prosjektet, slik at du får følgende path - `src/flatgeobuf/data`.

5. Pakk ut innholdet fra zip-filen i den nye `data` mappen. Da skal pathen til mappen se slik ut - `src\flatgeobuf\data\Basisdata_0000_Norge_25833_N50Kartdata_FGDB.gdb`

---

### Steg 2: Last ned nødvendige Python-pakker
For å utføre konverteringsprosessen trenger vi et par python-pakker. 
Disse har følgende bruksområder:
- Geopandas - for håndtering av geografiske data.
- Fiona - for lesing av gdb filer.
- Pyogrio - rask I/O for geografiske data.

In [ ]:
# Installer nødvendige pakker
import geopandas as gpd
import fiona  

# Path - for enkel filhåndtering
from pathlib import Path

---

### Steg 3: List ut lag fra N50 datasett
Vi benytter funksjonen `list_gdb_layers` til å liste ut alle de tilgjengelige lagene som følger med N50 datasettet.

In [3]:
def list_gdb_layers(gdb_path):
    """
    List alle tilgjengelige lag i en GDB-fil.
    
    Args:
        gdb_path (str): Path til GDB-filen.
        
    Returns:
        list: Liste over lagnavn i GDB-filen.
    """
    try:
        layers = fiona.listlayers(str(gdb_path))
        print("\nTilgjengelige lag i GDB:")
        for idx, layer in enumerate(layers, 1):
            print(f"{idx}. {layer}")
        return layers
    except Exception as e:
        print(f"Feil ved listing av lag: {e}")
        return []

La oss kjøre funksjonen og se hvilke lag vi har tilgjengelig, og hvor mange som finnes i datasettet.

In [ ]:
# Sett path til GDB-mappen
gdb_path = Path("data/Basisdata_0000_Norge_25833_N50Kartdata_FGDB.gdb")

if not gdb_path.exists():
    print(f"Feil: Fant ikke GDB-fil på {gdb_path}")
else:
    # List ut alle lag i GDB-filen
    layers = list_gdb_layers(gdb_path)
    print(f"\nTotalt antall lag funnet: {len(layers)}")


---

### Steg 4: Utfør konverteringsoppgave
Vi benytter funksjonen `convert_layer_to_fgb` til å konvertere et gdb-lag til en fgb-fil. Her gjennbruker vi `gdb_path` fra tidligere, vi velger hvilket lag vi ønsker å konvertere med `layer_name`, og velger at den ferdige fgb-filen lander i en `output` mappe.

In [ ]:
def convert_layer_to_fgb(gdb_path: Path, layer_name: str, output_dir: Path) -> Path:
    """
    Konverter et enkelt lag fra GDB til FlatGeobuf-format.
    
    Args:
        gdb_path (Path): Path til GDB-filen.
        layer_name (str): Navn på laget som skal konverteres.
        output_dir (Path): Mappe for lagring av resultatet.
        
    Returns:
        Path: Path til opprettet FlatGeobuf-fil.
    """
    try:
        output_dir.mkdir(parents=True, exist_ok=True)
        output_file = output_dir / f"{layer_name}.fgb"
        
        print(f"\nKonverterer lag: {layer_name}")
        print(f"Leser fra: {gdb_path}")
        print(f"Skriver til: {output_file}")
        
        print(f"\nLeser fra GDB... For store lag kan dette ta litt tid.")
        
        gdf = gpd.read_file(gdb_path, layer=layer_name, engine="pyogrio")
        
        # Vis info om laget
        print(f"\nLaginformasjon:")
        print(f"Antall objekter: {len(gdf)}")
        print(f"Geometri-type: {gdf.geometry.type.iloc[0]}")
        print(f"Koordinatreferansesystem (CRS): {gdf.crs}")

        print(f"\nSkriver til FlatGeobuf-format... For store lag kan dette også ta litt tid.")

        gdf.to_file(output_file, driver="FlatGeobuf")
        
        print(f"\nKonvertering fullført!")
        return output_file
        
    except Exception as e:
        print(f"Feil under konvertering av lag: {e}")
        return None


La oss kjøre konverteringsfunksjonen. Her har vi tatt utgangspunkt i lage #6 `N50_BygningerOgAnlegg_omrade`, men du kan endre dette til et av de andre lagene fra listen vi fikk tidligere.

In [ ]:
# Velg lag som skal konverteres (juster etter behov)
layer_name = "N50_BygningerOgAnlegg_omrade"

# Sett path til output mappen
output_dir = Path("output")

# Konverter valgt lag til FGB
output_file = convert_layer_to_fgb(gdb_path, layer_name, output_dir)


---

### Steg 5: Se resultatet på kartet

Gratulerer, du har nå generet en ny FlatGeobuf-fil!

La oss sjekke ut resultatet ved å få det opp på kartet. For å spare litt på maskinkraft, tar vi utgangspunkt i å begrense kartvisningen og dataene til et bestemt området, Kristiansand.
Her benytter vi funksjonen `visualize_area_on_map` til å laste inn fgb-dataene og leser det ut på et interaktivt kart.


In [19]:
from shapely.geometry import Point

def visualize_area_on_map(fgb_path: Path, center_point: tuple, buffer_km: float = 10):
    """
    Visualiserer et område fra en FlatGeobuf-fil på et interaktivt kart.
    
    Args:
        fgb_path (Path): Path til FlatGeobuf-filen.
        center_point (tuple): Tuple med (longitude, latitude) for senterpunktet.
        buffer_km (float): Radius i kilometer for området som skal vises (default: 10).
        
    Returns:
        folium.Map: Interaktivt kart med visualiserte data.
    """
    try:
        # Les inn data fra FlatGeobuf fil
        print(f"Leser data fra {fgb_path}...")
        gdf = gpd.read_file(fgb_path, driver="FlatGeobuf")
        
        # Opprett senterpunkt og konverter til riktig CRS
        lon, lat = center_point
        center_gdf = gpd.GeoDataFrame(
            geometry=[Point(lon, lat)], 
            crs="EPSG:4326"
        ).to_crs(gdf.crs)
        
        # Filtrer data innenfor buffer (konverter km til meter)
        buffer_meters = buffer_km * 1000
        gdf_filtered = gdf[gdf.intersects(center_gdf.geometry.iloc[0].buffer(buffer_meters))]
        gdf_filtered = gdf_filtered.to_crs("EPSG:4326")
        bounds = gdf_filtered.total_bounds
        
        print(f"Fant {len(gdf_filtered)} objekter i området")
        
        # Lag visuelt kart
        map = gdf_filtered.explore(
            tiles="CartoDB.Positron",
            style_kwds={"color": "blue", "weight": 1, "fillOpacity": 0.4},
            location=[lat, lon],
            zoom_start=13,
            max_bounds=True,
            min_zoom=12,
            max_zoom=15,
            min_lat=bounds[1] - 0.05,
            max_lat=bounds[3] + 0.05,
            min_lon=bounds[0] - 0.05,
            max_lon=bounds[2] + 0.05,
        )
        
        return map
        
    except Exception as e:
        print(f"Feil under visualisering av kart: {e}")
        return None

Velg hvilke FGB-fil du ønsker å vise på kartet. 

*OBS. - store filer kan bruke litt tid på å komme opp på kartet*

In [ ]:
# Definer parametre for Kristiansand
kristiansand_center = (8.0167, 58.15)  # longitude, latitude
# Velg hvilke FGB-fil du ønsker å vise på kartet. 
fgb_file = Path("output/N50_BygningerOgAnlegg_omrade.fgb")

# Kjør funksjonen og vis kartet
map = visualize_area_on_map(fgb_file, kristiansand_center)
map

---

## Videre bruk

Den konverterte `.fgb`-filen kan nå brukes direkte i f.eks. QGIS, web-apper (MapLibre GL, Deck.gl, etc.), eller enkelt leses inn i Python, JavaScript, eller andre miljøer med støtte for FlatGeobuf.

Dette gjør det lettere for utviklere å jobbe med åpne, norske kartdata!
